In [ ]:
!pip install -q -U google-genai

In [ ]:
!pip install groq

### LLM interactions:
Closed-Source
- OpenAi
- Anthropic
- Gemini

  
Open-Source
- Ollama
- Hugging Face
- Groq

In [ ]:
import os 
from dotenv import load_dotenv
from openai import OpenAI
import ollama
import anthropic
import google.generativeai
from groq import Groq
from google import genai

In [ ]:
#constants 
MODEL_GPT = "gpt-4o-mini"
MODEL_LLAMA= "llama3.2"
MODEL_ANTHROPIC = "claude-3-5-haiku-20241022"
MODEL_GEMINI = "gemini-2.0-flash"

In [ ]:
#set up environment
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")
claude_api_key= os.getenv("ANTHROPIC_API_KEY")
google_api_key=os.getenv("GEMINI_API_KEY")

#Check key works
if not openai_api_key:
    print ("No OpenAi API key found")
elif not openai_api_key.startswith("sk-proj-"):
    print("OpenAi API found but does not start with sk-proj-")
elif openai_api_key.strip() != openai_api_key:
    print("OpenAI API found but might contain space or tab character - remove them")
else:
    print("OpenAi API found and looks good")

In [ ]:
#we create an instance of models
openai=OpenAI()
groq_model = Groq()
claude = anthropic.Anthropic()
gemini = genai.Client(api_key=google_api_key)

In [ ]:

#OpenAI structure
system_prompt= "You are a helpful assistant"
user_prompt = "Explain embedding in simple words"

message =[{"role":"system","content":system_prompt},{"role":"user","content":user_prompt}]

result = openai.chat.completions.create(model=MODEL_GPT, messages=message)
print(result.choices[0].message.content)

In [ ]:
#Ollama
result_ollama = ollama.chat(model=MODEL_LLAMA,messages=message)
print(result_ollama["message"]["content"])

In [ ]:
#Groq Structure uses any model ontop 
resul_groq = groq_model.chat.completions.create(model="llama-3.3-70b-versatile",messages=message)
print(resul_groq.choices[0].message.content)

In [ ]:
#Anthropic Structure 
#Anthropic needs SYSTEM_message separate from USER_message
#also needs Max tokens
claude_message = claude.messages.create(
    model=MODEL_ANTHROPIC,
    max_tokens=200,
    temperature=0.7,
    system=system_prompt,
    messages=[{"role":"user","content":user_prompt}]
)
print(claude_message.content[0].text)

In [ ]:
#Google Gemini Structure
gemini_result = gemini.models.generate_content(model= MODEL_GEMINI,contents=user_prompt)
print(gemini_result.text)

### Streamming 

In [ ]:
#OpenAI Streamming
result = openai.chat.completions.create(model=MODEL_GPT, messages=message, stream=True)

for chunk in result:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")


In [ ]:
#Anthropic Streamming
#NOTRE: claude.message.stream() 
claude_message = claude.messages.stream(
    model=MODEL_ANTHROPIC,
    max_tokens=200,
    temperature=0.7,
    system=system_prompt,
    messages=[{"role":"user","content":user_prompt}]
)

with claude_message as stream:
    for text in stream.text_stream:
        print(text, end="",flush=True)


In [ ]:
#Ollama Streamming
result_ollama = ollama.chat(model=MODEL_LLAMA,messages=message, stream= True)
for chunk in result_ollama:
     # end='' prevents extra newlines
    print(chunk["message"]["content"],end="")


In [ ]:
#Gemini Streamming
#Google recently announced endpoints compatible with OpenAI's python client library

gemini_via_openai_client = OpenAI(
    api_key=google_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

stream = gemini_via_openai_client.chat.completions.create(
    model=MODEL_GEMINI,
    messages=message,
    stream=True
)

for chunk in stream:
    print(chunk.choices[0].delta.content,end="")
